# Datos centrales de carbón arrancadas

Los datos de programación diaria de generación a nivel de central (UPROG) no se publican en la web pública de esios a diario, si no en la web de agentes, en el fichero P48_cierre. El dato se hace público al 90 día transcurrido.

Mi empresa, como agentes de mercado, dispone de esta información a diario y la almacenamos en sql. A través del siguiente código MySql, he descargado los datos calculando cuántas centrales había arrancadas cada día en cada hora y posteriormente lo he guardado en el .csv adjunto en el repositorio en el apartado datos.

~~~~sql
SELECT
      COUNT(DISTINCT(id_uprog)) AS NUM_PLANTAS
      ,format([FECHASESION],'yyyy-MM-dd') + ' ' + format([HORA]-1,'00') + ':00:00' AS datetime
      FROM [BBDD]  -- representa la bbdd de nuestro servidor
      where id_uprog in ('','',...)  -- lista de unidades. Existen 26 centrales de carbón activas en España
and fechasesion between '20150930' and '20180930'

GROUP BY datetime
 
~~~~

En este notebook leo los datos del .csv y los cargo en un nuevo dataframe.

## 1. Importamos librerías necesarias

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

## 2. Lectura y testeo de datos

In [2]:
carbon=pd.read_csv("../data/P48.csv",sep=";")
carbon.head() 

,datetime,NUM_PLANTAS
0,30/9/15 0:00,22
1,30/9/15 1:00,22
2,30/9/15 2:00,22
3,30/9/15 3:00,23
4,30/9/15 4:00,23


Como feature no quiero las centrales de carbón arrancadas en el día, ya que no es un dato que tenga y pueda utilizar como entrada para predecir la subida/bajada del precio del intradiario (cuando debo apostar todavía no sé la casación ni qué centrales estarán).

Lo que utilizaré como input será qué centrales había arrancadas el día anterior. El hecho de que una central ya esté arrancada puede condicionar el comportamiento del intradiario al día siguiente en función del precio del MD resultante.

Por tanto, sumaré un día a datetime, de tal manera que para el día 01/10/2015 el dato que me muestre sea el nº de centrales de carbón arrancadas el 30/09/2015.

In [3]:
carbon.dtypes

datetime       object
NUM_PLANTAS     int64
dtype: object

In [4]:
carbon['datetime']=pd.to_datetime(carbon['datetime'],dayfirst=True)

In [5]:
#Unificamos formato

carbon['datetime']=[x.strftime('%Y-%m-%d %H:%M:%S') for x in carbon['datetime']]

In [6]:
pd.to_datetime?

Signature: pd.to_datetime(arg, errors='raise', dayfirst=False, yearfirst=False, utc=None, box=True, format=None, exact=True, unit=None, infer_datetime_format=False, origin='unix', cache=False)
Docstring:
Convert argument to datetime.

Parameters
----------
arg : integer, float, string, datetime, list, tuple, 1-d array, Series

    .. versionadded:: 0.18.1

       or DataFrame/dict-like

errors : {'ignore', 'raise', 'coerce'}, default 'raise'

    - If 'raise', then invalid parsing will raise an exception
    - If 'coerce', then invalid parsing will be set as NaT
    - If 'ignore', then invalid parsing will return the input
dayfirst : boolean, default False
    Specify a date parse order if `arg` is str or its list-likes.
    If True, parses dates with the day first, eg 10/11/12 is parsed as
    2012-11-10.
    with day first (this is a known bug, based on dateutil behavior).
yearfirst : boolean, default False
    Specify a date parse order if `arg` is str or its list-likes.

    - If T

In [9]:
carbon['datetime']=pd.to_datetime(carbon['datetime'])
carbon.dtypes

datetime       datetime64[ns]
NUM_PLANTAS             int64
dtype: object

In [10]:
dia=timedelta(days=1)  #nº días que vamos a sumar para crear nuestra columna adicional

In [11]:
carbon['aux']=[x+dia for x in carbon['datetime']]
carbon.head()

,datetime,NUM_PLANTAS,aux
0,2015-09-30 00:00:00,22,2015-10-01 00:00:00
1,2015-09-30 01:00:00,22,2015-10-01 01:00:00
2,2015-09-30 02:00:00,22,2015-10-01 02:00:00
3,2015-09-30 03:00:00,23,2015-10-01 03:00:00
4,2015-09-30 04:00:00,23,2015-10-01 04:00:00


In [12]:
carbon.tail()

,datetime,NUM_PLANTAS,aux
26323,2018-09-30 19:00:00,13,2018-10-01 19:00:00
26324,2018-09-30 20:00:00,13,2018-10-01 20:00:00
26325,2018-09-30 21:00:00,13,2018-10-01 21:00:00
26326,2018-09-30 22:00:00,13,2018-10-01 22:00:00
26327,2018-09-30 23:00:00,13,2018-10-01 23:00:00


Las últimas 24 horas me sobran, mi histórico de datos (columna aux) la he parado en 30/09/2018. Los últimos 24 datos serían los que ayudarían a predecir el mercado del día 01/10/2018.

In [13]:
carbon_data=carbon[['aux','NUM_PLANTAS']]

In [14]:
ult_dia=pd.to_datetime('2018-10-01 00:00')

carbon_data2 = carbon_data[(carbon_data['aux']<ult_dia)]

In [15]:
carbon_data2.shape

(26304, 2)

In [16]:
carbon_data2[carbon_data2['aux'].duplicated()==True]

,aux,NUM_PLANTAS
603,2015-10-26 02:00:00,18
9507,2016-10-31 02:00:00,19
18243,2017-10-30 02:00:00,10


Al haber añadido 1 día, los días de 25 horas no me van a coincidir con los reales. Tendré que tenerlo en cuenta a la hora de unificar datos eliminando por separado duplicados.

Lo mismo pasará con los de 23 h. Me surgirán NaN en ambos historicos

In [17]:
carbon_dia_ant=carbon_data2.rename(columns={'aux':'datetime','NUM_PLANTAS':'NUM_PLANTAS_D-1'})

In [18]:
carbon_dia_ant.set_index('datetime',inplace=True)

In [19]:
carbon_dia_ant.head()

,NUM_PLANTAS_D-1
datetime,
2015-10-01 00:00:00,22
2015-10-01 01:00:00,22
2015-10-01 02:00:00,22
2015-10-01 03:00:00,23
2015-10-01 04:00:00,23


In [20]:
carbon_dia_ant.shape  #sin NaN

(26304, 1)

In [21]:
carbon_dia_ant['NUM_PLANTAS_D-1'].count()

26304

## 3. Guardamos en csv

In [52]:
carbon_dia_ant.to_csv('../data/carbon_dia_ant.csv',sep=';')